<a href="https://colab.research.google.com/github/nrrajuu/Machine-Learning/blob/main/House%20Price%20Prediction%20using%20Machine%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## First let's import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math
from scipy.stats import norm, skew
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
train.shape , test.shape

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
# Our target column is 'SalePrice'. So let's analyse it-
train['SalePrice'].describe()

In [ ]:
 sns.distplot(train['SalePrice'])

In [ ]:
print("Skewness: %f" % train['SalePrice'].skew())
print("Kurtosis: %f" % train['SalePrice'].kurt())

In [ ]:
train['SalePrice'] = np.log1p(train['SalePrice'])
sns.distplot(train['SalePrice'], fit=norm);

In [ ]:
sns.boxplot(data=train,x='OverallQual',y='SalePrice')

In [ ]:
plt.figure(figsize=(15,6),dpi=130)
sns.boxplot(data=train,x='YearBuilt',y='SalePrice')
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.scatterplot(data=train,x='TotalBsmtSF',y='SalePrice')

In [ ]:
sns.scatterplot(data=train,x='GrLivArea',y='SalePrice')

In [ ]:
# Let's check which features are the most corelated
train.corr()["SalePrice"].sort_values(ascending=False)[1:]

In [ ]:
corrmat = train.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
corr = train.corr()
highest_corr_features = corr.index[abs(corr["SalePrice"])>0.5]
plt.figure(figsize=(10,10))
g = sns.heatmap(train[highest_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(train[cols])

In [ ]:
y_train = train['SalePrice']
test_id = test['Id']
all_data = pd.concat([train, test], axis=0, sort=False)
all_data = all_data.drop(['Id', 'SalePrice'], axis=1)

In [ ]:
Total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum() / all_data.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([Total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(25)

In [ ]:
all_data.drop((missing_data[missing_data['Total'] > 5]).index, axis=1, inplace=True)
print(all_data.isnull().sum().max())

In [ ]:
total = all_data.isnull().sum().sort_values(ascending=False)
total.head(19)

In [ ]:
# filling the numeric data
numeric_missed = ['BsmtFinSF1',
                  'BsmtFinSF2',
                  'BsmtUnfSF',
                  'TotalBsmtSF',
                  'BsmtFullBath',
                  'BsmtHalfBath',
                  'GarageArea',
                  'GarageCars']

for feature in numeric_missed:
    all_data[feature] = all_data[feature].fillna(0)

In [ ]:
#filling categorical data
categorical_missed = ['Exterior1st',
                  'Exterior2nd',
                  'SaleType',
                  'MSZoning',
                   'Electrical',
                     'KitchenQual']

for feature in categorical_missed:
    all_data[feature] = all_data[feature].fillna(all_data[feature].mode()[0])

In [ ]:
#Fill in the remaining missing values with the values that are most common for this feature.

all_data['Functional'] = all_data['Functional'].fillna('Typ')

In [ ]:
all_data.drop(['Utilities'], axis=1, inplace=True)

In [ ]:
all_data.isnull().sum().max() #just checking that there's no missing data missing..

In [ ]:
numeric_feats = all_data.dtypes[all_data.dtypes != 'object'].index
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_feats[abs(skewed_feats) > 0.5]
high_skew

In [ ]:
for feature in high_skew.index:
    all_data[feature] = np.log1p(all_data[feature])

In [ ]:
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [ ]:
all_data = pd.get_dummies(all_data)
all_data.head()

In [ ]:
x_train =all_data['LotArea'][:len(y_train)].values
x_test = all_data['LotArea'][len(y_train):]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=42)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
from sklearn.linear_model import LinearRegression
the_model = LinearRegression()
the_model.fit(X_train.reshape(-1,1), Y_train)

In [ ]:
y_pred=the_model.predict(X_test.reshape(-1,1))

In [ ]:
y_pred

In [ ]:
y_test=y_test.values

In [ ]:
y_test

In [ ]:
plt.scatter(x_train,y_train)
plt.xlabel('LotArea')
plt.ylabel('House Price')
plt.title('Visualization of LotArea Vs House Price')

In [ ]:
plt.scatter(X_test,y_pred,label='prediction')
plt.scatter(X_test,y_test,label='Orginal')
plt.xlabel('LotArea')
plt.ylabel('House Price')
plt.title('Visualization of Predicted and Orginal Values')